# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [2]:
# import the train dataset ( 1000 first rows)
import pandas as pd
df = pd.read_csv('../train.csv',sep=',',nrows=1_000)

In [3]:
# prepare X and y
X = df.drop('fare_amount', axis=1)
y = df.fare_amount

In [4]:
# Hold out ( train and test dplit )
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
29,2013-08-11 00:52:00.00000026,2013-08-11 00:52:00 UTC,-73.981020,40.737760,-73.980668,40.730497,2
535,2013-01-13 13:38:00.000000119,2013-01-13 13:38:00 UTC,-73.989647,40.747290,-73.985648,40.735408,5
695,2013-11-24 20:54:00.000000162,2013-11-24 20:54:00 UTC,-73.971542,40.750487,-73.988967,40.729940,1
557,2010-02-03 20:51:29.0000003,2010-02-03 20:51:29 UTC,-73.954191,40.764029,-73.918043,40.766876,1
836,2012-03-16 07:52:00.000000155,2012-03-16 07:52:00 UTC,-73.960797,40.818232,-73.953255,40.810163,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [6]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [8]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon


    def fit(self, X, y=None):
        return self


    def transform(self, X, y=None):
        X_ = X.copy()
        X_["distance"] = haversine_vectorized(X_, 
                                    self.start_lat, self.start_lon,
                                    self.end_lat, self.end_lon)
        return X_[['distance']]

In [9]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
29,0.808153
535,1.363497
695,2.715720
557,3.060721
836,1.099034


In [12]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self,time_col ='pickup_datetime', timezone = 'America/New_York'):
       self.time_col = time_col
       self.timezone = timezone


    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        X_[self.time_col] = pd.to_datetime(X_[self.time_col], format="%Y-%m-%d %H:%M:%S %Z").dt.tz_convert(self.timezone)
        X_['dow']=X_[self.time_col].dt.dayofweek
        X_['hour'] = X_[self.time_col].dt.hour
        X_['month'] = X_[self.time_col].dt.month
        X_['year'] = X_[self.time_col].dt.year
        X_ = X_.set_index(self.time_col)
        
        return X_[['dow', 'hour']]

In [13]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour
pickup_datetime,,
2013-08-10 20:52:00-04:00,5,20
2013-01-13 08:38:00-05:00,6,8
2013-11-24 15:54:00-05:00,6,15
2010-02-03 15:51:29-05:00,2,15
2012-03-16 03:52:00-04:00,4,3


###  Prepocessing pipeline

In [14]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [15]:

# create distance pipeline dist_pipe
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# create distance pipeline dist_pipe
dist_pipe = Pipeline([('dist_transformer',DistanceTransformer()),('std_scaler',StandardScaler())])

# display distance pipeline
dist_pipe

Pipeline(steps=[('dist_transformer', DistanceTransformer()),
                ('std_scaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [16]:
from sklearn.preprocessing import OneHotEncoder
# create time pipeline time_pipe
# time_cat_transformer = OneHotEncoder()
time_pipe = Pipeline([('time_encoder',TimeFeaturesEncoder()),('one_hot',OneHotEncoder(handle_unknown="ignore"))])

# display time pipeline
time_pipe

Pipeline(steps=[('time_encoder', TimeFeaturesEncoder()),
                ('one_hot', OneHotEncoder(handle_unknown='ignore'))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [17]:
from sklearn.compose import ColumnTransformer
# create preprocessing pipeline preproc_pipe
Preprocessor = ColumnTransformer([
    ('time_pipe',time_pipe,['pickup_datetime']),
    ('dist_pipe',dist_pipe,['pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude'])
    ])
# display preprocessing pipeline
Preprocessor

ColumnTransformer(transformers=[('time_pipe',
                                 Pipeline(steps=[('time_encoder',
                                                  TimeFeaturesEncoder()),
                                                 ('one_hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime']),
                                ('dist_pipe',
                                 Pipeline(steps=[('dist_transformer',
                                                  DistanceTransformer()),
                                                 ('std_scaler',
                                                  StandardScaler())]),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude', 'dropoff_latitude'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [18]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
pipe = Pipeline([
    ('preprocessing',Preprocessor),
    ('linear_regression',LinearRegression())

])
# display the pipeline with model
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('time_pipe',
                                                  Pipeline(steps=[('time_encoder',
                                                                   TimeFeaturesEncoder()),
                                                                  ('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime']),
                                                 ('dist_pipe',
                                                  Pipeline(steps=[('dist_transformer',
                                                                   DistanceTransformer()),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['pickup_longitude',
                                                   'pickup_latitude',
                                                   'dropoff_longitude',
                                                   'dropoff_latitude'])])),
                ('linear_regression', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [22]:
# train the pipelined model
pipe_trained = pipe.fit(X_train,y_train)


# compute y_pred on the test set
y_pred = pipe_trained.predict(X_test)

Use the RMSE to evaluate the performance of the model:

In [20]:
def compute_rmse(y_pred, y_true):
    
    return np.sqrt(((y_pred - y_true) ** 2).mean()) # A COMPLETER

In [23]:
# call compute_rmse
compute_rmse(y_pred, y_test)

8.687247165178421

In [24]:
pipe_trained.score(X_test, y_test)

-0.08617711902031644

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [31]:
# implement get_data() function
def get_data(nrows=10000):
    '''returns a DataFrame with nrows from s3 bucket'''
    df = pd.read_csv('../train.csv',sep=',',nrows=nrows)
    return df

In [26]:
# implement clean_data() function
def clean_data(df, test=False):
    df = df[
        (df.fare_amount > 0) &
        (df.passenger_count <= 8) &
        (df.passenger_count > 0)&
        (df["pickup_latitude"].between(left = 40, right = 42 ))&
        (df["pickup_longitude"].between(left = -74.3, right = -72.9 ))&
        (df["dropoff_latitude"].between(left = 40, right = 42 ))&
        (df["dropoff_longitude"].between(left = -74, right = -72.9 ))

        ]
    return df

In [27]:
# implement set_pipeline() function
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

def set_pipeline():
    dist_pipe = Pipeline([('dist_transformer',DistanceTransformer()),('std_scaler',StandardScaler())])
    time_pipe = Pipeline([('time_encoder',TimeFeaturesEncoder()),('one_hot',OneHotEncoder(handle_unknown="ignore"))])

    Preprocessor = ColumnTransformer([
        ('time_pipe',time_pipe,['pickup_datetime']),
        ('dist_pipe',dist_pipe,['pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude'])
        ])

    pipe = Pipeline([
        ('preprocessing',Preprocessor),
        ('linear_regression',LinearRegression())
        ])
    
    return pipe

In [28]:
# implement train() function
def train(X_train, y_train, pipeline):
    pipe_trained = pipeline.fit(X_train,y_train)
    return pipe_trained

In [29]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    y_pred = pipeline.predict(X_test)
    rmse = np.sqrt(((y_pred - y_test) ** 2).mean())
    print(rmse)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [32]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

11.401003483072019


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀